In [18]:
import pandas as pd 
import numpy as np 
import os

In [2]:
##Reading the data
matches_data = pd.read_csv('data/matches.csv')
deliveries_data = pd.read_csv('data/deliveries.csv')

In [3]:
matches_data.head(5)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [4]:
##Getting column names
matches_data.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [5]:
## Using only required columns
final_matches_data = matches_data[['id', 'season', 'city','venue']]

In [6]:
deliveries_data.head(5)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [7]:
## Column names
deliveries_data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [8]:
print("Total Records :", deliveries_data.shape[0])
print("Records with super over : ",deliveries_data[deliveries_data['is_super_over'] == 1].shape[0])

Total Records : 179078
Records with super over :  81


In [9]:
## Removing records with Super over
super_over_rowIds = deliveries_data[deliveries_data['is_super_over'] == 1].index
deliveries_data.drop(labels= super_over_rowIds,axis=0,inplace=True)

## Removing records with innings 3, 4 and 5
deliveries_data = deliveries_data[deliveries_data['inning'].isin([1,2])]

## Dropping columns 
drop_cols = ['batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs', 'bye_runs', 'legbye_runs', 
             'noball_runs','penalty_runs','batsman_runs', 'extra_runs','dismissal_kind', 'fielder']

deliveries_data.drop(labels=drop_cols, axis=1, inplace =True)

In [10]:
deliveries_data.shape

(178982, 8)

In [11]:
## finding cummulative total and wickets lost
deliveries_data['Cumsum_Total'] = deliveries_data.groupby(by=['match_id', 'inning'])['total_runs'].apply(lambda x : x.cumsum())
#
deliveries_data['wickets_lost'] = np.where(deliveries_data['player_dismissed'].isnull(),0,1)
deliveries_data['total_wickets'] = deliveries_data.groupby(by=['match_id', 'inning'])['wickets_lost'].apply(lambda x : x.cumsum())

## Runs scored and wicket lost in last 5 overs


In [12]:
deliveries_data.groupby(['match_id', 'inning'],as_index=False).last()

,match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed,Cumsum_Total,wickets_lost,total_wickets
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,20,6,6,Yuvraj Singh,207,0,4
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,4,0,YS Chahal,172,1,10
2,2,1,Mumbai Indians,Rising Pune Supergiant,20,7,1,TG Southee,184,0,8
3,2,2,Rising Pune Supergiant,Mumbai Indians,20,5,6,BA Stokes,187,0,3
4,3,1,Gujarat Lions,Kolkata Knight Riders,20,6,4,KD Karthik,183,0,4
...,...,...,...,...,...,...,...,...,...,...,...
1505,11413,2,Delhi Capitals,Sunrisers Hyderabad,20,6,4,A Mishra,170,0,7
1506,11414,1,Delhi Capitals,Chennai Super Kings,20,6,6,TA Boult,155,0,9
1507,11414,2,Chennai Super Kings,Delhi Capitals,19,6,8,MS Dhoni,162,0,4
1508,11415,1,Mumbai Indians,Chennai Super Kings,20,6,4,MJ McClenaghan,152,0,8


In [13]:
for i in range(30):
    deliveries_data['Lag_Runs_'+str(i+1)] = deliveries_data.groupby(by=['match_id', 'inning'])['total_runs'].shift(i).fillna(0)
    deliveries_data['Lag_Wickets_'+str(i+1)] = deliveries_data.groupby(by=['match_id', 'inning'])['wickets_lost'].shift(i).fillna(0)

## Last 5overs runs
col_lagruns = deliveries_data.filter(regex='Lag_Runs_').columns
deliveries_data['Last_5overs_runs'] = deliveries_data[col_lagruns].sum(axis=1)
deliveries_data.drop(labels=col_lagruns,axis=1,inplace=True)

## Last 5overs wickets
col_lagWkts = deliveries_data.filter(regex='Lag_Wickets_').columns
deliveries_data['Last_5overs_wickets'] = deliveries_data[col_lagWkts].sum(axis=1)
deliveries_data.drop(labels=col_lagWkts,axis=1,inplace=True)

In [14]:
deliveries_data.groupby(by=['match_id', 'inning'],as_index=False).last()

,match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed,Cumsum_Total,wickets_lost,total_wickets,Last_5overs_runs,Last_5overs_wickets
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,20,6,6,Yuvraj Singh,207,0,4,56.0,1.0
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,4,0,YS Chahal,172,1,10,40.0,5.0
2,2,1,Mumbai Indians,Rising Pune Supergiant,20,7,1,TG Southee,184,0,8,59.0,3.0
3,2,2,Rising Pune Supergiant,Mumbai Indians,20,5,6,BA Stokes,187,0,3,48.0,1.0
4,3,1,Gujarat Lions,Kolkata Knight Riders,20,6,4,KD Karthik,183,0,4,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,11413,2,Delhi Capitals,Sunrisers Hyderabad,20,6,4,A Mishra,170,0,7,55.0,3.0
1506,11414,1,Delhi Capitals,Chennai Super Kings,20,6,6,TA Boult,155,0,9,54.0,4.0
1507,11414,2,Chennai Super Kings,Delhi Capitals,19,6,8,MS Dhoni,162,0,4,41.0,2.0
1508,11415,1,Mumbai Indians,Chennai Super Kings,20,6,4,MJ McClenaghan,152,0,8,47.0,3.0


In [15]:
team_score_data = deliveries_data.groupby(by=['match_id', 'inning'],as_index=False)['Cumsum_Total'].max()
team_score_data.rename(columns={"Cumsum_Total":"Total_Score"},inplace=True)

In [16]:
## Merging with team score data and final matches data
final_score_data = deliveries_data.merge(team_score_data,how='left',on=['match_id', 'inning'])
final_score_data = final_score_data.merge(final_matches_data,how='left',left_on='match_id',right_on='id')

In [17]:
final_score_data.to_csv("data/final_score_data.csv",index=False)